In [134]:
import numpy as np
Q1 = np.zeros(((720)*(480),4))
Q2 = np.zeros(((720)*(480),4))
Q3 = np.zeros(((720)*(480),4))
Q4 = np.zeros(((720)*(480),4))

In [138]:

def loop_game():
    """
    Snake Eater
    Made with PyGame
    """
    iter = 0
    import pygame, sys, time, random
    import numpy as np
    from random import randint
    from scipy.spatial import distance

    # Difficulty settings
    # Easy      ->  10
    # Medium    ->  25
    # Hard      ->  40
    # Harder    ->  60
    # Impossible->  120
    difficulty = 120

    # Window size
    frame_size_x = 720
    frame_size_y = 480

    # Checks for errors encountered
    check_errors = pygame.init()
    # pygame.init() example output -> (6, 0)
    # second number in tuple gives number of errors
    if check_errors[1] > 0:
        print(f'[!] Had {check_errors[1]} errors when initialising game, exiting...')
        sys.exit(-1)
    else:
        print('[+] Game successfully initialised')


    # Initialise game window
    pygame.display.set_caption('Snake Eater')
    game_window = pygame.display.set_mode((frame_size_x, frame_size_y))


    # Colors (R, G, B)
    black = pygame.Color(0, 0, 0)
    white = pygame.Color(255, 255, 255)
    red = pygame.Color(255, 0, 0)
    green = pygame.Color(0, 255, 0)
    blue = pygame.Color(0, 0, 255)


    # FPS (frames per second) controller
    fps_controller = pygame.time.Clock()


    # Game variables
    #snake_pos = [random.randint(0,710),random.randint(0,470)]
    snake_pos=[0,0]
    snake_body = [[100, 50], [100-10, 50], [100-(2*10), 50]]
    global prev_pos0_evol
    prev_pos0_evol = 0
    global prev_pos1_evol
    prev_pos1_evol = 0

    food_pos = [random.randrange(1, (frame_size_x//10)) * 10, random.randrange(1, (frame_size_y//10)) * 10]
    #food_pos=[600,200]
    food_spawn = True

    direction = 'RIGHT'
    change_to = direction

    score = 0


    # Game Over
    def game_over():
        my_font = pygame.font.SysFont('times new roman', 90)
        game_over_surface = my_font.render('YOU DIED', True, red)
        game_over_rect = game_over_surface.get_rect()
        game_over_rect.midtop = (frame_size_x/2, frame_size_y/4)
        game_window.fill(black)
        game_window.blit(game_over_surface, game_over_rect)
        show_score(0, red, 'times', 20)
        pygame.display.flip()
        time.sleep(3)
        loop_game()
        #pygame.quit()
        #sys.exit()


    # Score
    def show_score(choice, color, font, size):
        score_font = pygame.font.SysFont(font, size)
        score_surface = score_font.render('Score : ' + str(score), True, color)
        score_rect = score_surface.get_rect()
        if choice == 1:
            score_rect.midtop = (frame_size_x/10, 15)
        else:
            score_rect.midtop = (frame_size_x/2, frame_size_y/1.25)
        game_window.blit(score_surface, score_rect)
        # pygame.display.flip()

    
    def take_action(st, Q, eps, prev_pos0_evol, prev_pos1_evol):
        # Take an action
        if random.uniform(0, 1) < eps:
            action = randint(0, 3)
        else: # Or greedy action
            l= [0,0]
            z = np.array(Q[st])
            l[0]=np.argmin(z)
            z[np.argmin(z)]=+10
            l[1]=np.argmin(z)
            a = (food_pos[0],food_pos[1])
            dist = np.zeros((4,1))
            dist[0] = distance.euclidean((food_pos[0],food_pos[1]), (snake_pos[0],snake_pos[1]-10))/10000
            dist[1] = distance.euclidean((food_pos[0],food_pos[1]), (snake_pos[0],snake_pos[1]+10))/10000
            dist[2] = distance.euclidean((food_pos[0],food_pos[1]), (snake_pos[0]-10,snake_pos[1]))/10000
            dist[3] = distance.euclidean((food_pos[0],food_pos[1]), (snake_pos[0]+10,snake_pos[1]))/10000
            if sum(Q[st]) != 0:
                dist[l[0]] = 10000
                #dist[l[1]] = 10000
            #print(dist,snake_pos,food_pos)
            #action = np.argmax(Q[st])
            action = np.argmin(dist)
            print(Q[st], st)
            cpt = 0
            while cpt < 3:
                if action == 1 and direction != 'UP':
                    snake_pos1_evol = +10
                    snake_pos0_evol = 0
                elif action == 0 and direction != 'DOWN':
                    snake_pos1_evol = -10
                    snake_pos0_evol = 0
                elif action == 3 and direction != 'LEFT':
                    snake_pos0_evol = +10
                    snake_pos1_evol = 0
                elif action == 2 and direction != 'RIGHT':
                    snake_pos0_evol = -10
                    snake_pos1_evol = 0
                else:
                    snake_pos0_evol = prev_pos0_evol
                    snake_pos1_evol = prev_pos1_evol

                prev_pos0_evol = snake_pos0_evol
                prev_pos1_evol = snake_pos1_evol

                for block in snake_body[1:]:
                    if snake_pos[0] + snake_pos0_evol == block[0] and snake_pos[1] + snake_pos1_evol == block[1]:
                        dist[action] = 1
                        action = np.argmin(dist)
                        print('cpt', cpt)
                cpt += 1
            print(action)
                    
        return action, prev_pos0_evol, prev_pos1_evol
    
    
    
    # Main logic
    while True:
        #print(iter, end='\r')
        if iter>5000:
            difficulty=10
        random_choice = 0.1
        
        if direction == "UP":
            Q = Q1
        if direction == "DOWN":
            Q = Q2
        if direction == "LEFT":
            Q = Q3
        if direction == "RIGHT":
            Q = Q4

        if iter > 0:
            st = stp1
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            # Whenever a key is pressed down
            elif event.type == pygame.KEYDOWN:
                # W -> Up; S -> Down; A -> Left; D -> Right
                if event.key == pygame.K_UP or event.key == ord('w'):
                    change_to = 'UP'
                if event.key == pygame.K_DOWN or event.key == ord('s'):
                    change_to = 'DOWN'
                if event.key == pygame.K_LEFT or event.key == ord('a'):
                    change_to = 'LEFT'
                if event.key == pygame.K_RIGHT or event.key == ord('d'):
                    change_to = 'RIGHT'
                # Esc -> Create event to quit the game
                if event.key == pygame.K_ESCAPE:
                    pygame.event.post(pygame.event.Event(pygame.QUIT))
        
        if snake_pos[0]==0 and snake_pos[1]==0:
            st = 0
        if snake_pos[0]==0 and snake_pos[1]!=0:
            st = (snake_pos[1]/10)*(((frame_size_x-10)/10)+1)
        if snake_pos[0]!=0 and snake_pos[1]==0:
            st = (snake_pos[0]/10)
        if snake_pos[0]!=0 and snake_pos[1]!=0:
            st = (((frame_size_x)/10)*(snake_pos[1]/10))+((snake_pos[0]/10))
        
    
        change_to = take_action(int(st), Q, random_choice, prev_pos0_evol, prev_pos1_evol)[0]
        at = change_to


        if change_to == 0 and direction != 'DOWN':
            stp1 = st - (frame_size_x/10)
        else:
            stp1  = st + (frame_size_x/10)
        if change_to == 1 and direction != 'UP':
            stp1 = st + (frame_size_x/10)
        else:
            stp1 = st - (frame_size_x/10)
        if change_to == 2 and direction != 'RiGHT':
            stp1 = st - 1
        else:
            stp1 = st + 1
        if change_to == 3 and direction != 'LEFT':
            stp1 = st + 1
        else:
            stp1 = st - 1
        if change_to == 4 and direction=='UP':
            stp1 = st - (frame_size_x/10)
        elif change_to == 4 and direction=='DOWN':
            stp1 = st + (frame_size_x/10)
        elif change_to == 4 and direction=='LEFT':
            stp1 = st - 1
        elif change_to == 4 and direction=='RIGHT':
            stp1 = st + 1
        #print("s", stp1)
        #print("r", r)

        # Update Q function
        #print(stp1)
        atp1 = take_action(int(stp1), Q, 0, prev_pos0_evol, prev_pos1_evol)[0]
        #print(st,at)
        Q[int(st)][at] = Q[int(st)][at] + 0.1*(score + 0.9*Q[int(stp1)][atp1] - Q[int(st)][at])
        

        
        
        # Making sure the snake cannot move in the opposite direction instantaneously
        if change_to == 0 and direction != 'DOWN':
            direction = 'UP'
        if change_to == 1 and direction != 'UP':
            direction = 'DOWN'
        if change_to == 2 and direction != 'RIGHT':
            direction = 'LEFT'
        if change_to == 3 and direction != 'LEFT':
            direction = 'RIGHT'
        if change_to == 4:
            direction = direction
        
        # Moving the snake
        if direction == 'UP':
            snake_pos[1] -= 10
        if direction == 'DOWN':
            snake_pos[1] += 10
        if direction == 'LEFT':
            snake_pos[0] -= 10
        if direction == 'RIGHT':
            snake_pos[0] += 10
        

        # Snake body growing mechanism
        snake_body.insert(0, list(snake_pos))
        if snake_pos[0] == food_pos[0] and snake_pos[1] == food_pos[1]:
            score += 1
            food_spawn = False
        else:
            snake_body.pop()

        # Spawning food on the screen
        if not food_spawn:
            food_pos = [random.randrange(1, (frame_size_x//10)) * 10, random.randrange(1, (frame_size_y//10)) * 10]
        food_spawn = True

        # GFX
        game_window.fill(black)
        for pos in snake_body:
            # Snake body
            # .draw.rect(play_surface, color, xy-coordinate)
            # xy-coordinate -> .Rect(x, y, size_x, size_y)
            pygame.draw.rect(game_window, blue, pygame.Rect(pos[0], pos[1], 10, 10))

        # Snake food
        pygame.draw.rect(game_window, white, pygame.Rect(food_pos[0], food_pos[1], 10, 10))

        
        # Game Over conditions
        # Getting out of bounds
        if snake_pos[0] < 0 or snake_pos[0] > frame_size_x-10:
            score = score -10
            #print('OB', score)
            #print('pos',st)
            Q[int(st)][at] = Q[int(st)][at] + 0.1*(score + 0.9*Q[int(stp1)][atp1] - Q[int(st)][at])
            loop_game()
            #game_over()
        if snake_pos[1] < 0 or snake_pos[1] > frame_size_y-10:
            score = score -10
            #print('OB', score)
            #print('pos',st)
            Q[int(st)][at] = Q[int(st)][at] + 0.1*(score + 0.9*Q[int(stp1)][atp1] - Q[int(st)][at])
            loop_game()
            #game_over()
        # Touching the snake body
        for block in snake_body[1:]:
            if snake_pos[0] == block[0] and snake_pos[1] == block[1]:
                score = score -10
                #print('TB',score)
                #print('pos',st)
                Q[int(st)][at] = Q[int(st)][at] + 0.1*(score + 0.9*Q[int(stp1)][atp1] - Q[int(st)][at])
                loop_game()
                #game_over()
                
        
    
        #print(Q.sum())
        show_score(1, white, 'consolas', 20)
        # Refresh game screen
        iter += 1
        pygame.display.update()
        # Refresh rate
        fps_controller.tick(difficulty)

In [139]:
loop_game()

[+] Game successfully initialised
[0. 0. 0. 0.] 3395
0
[0. 0. 0. 0.] 3394
0
[0. 0. 0. 0.] 3323
0
[0. 0. 0. 0.] 3322
0
[0. 0. 0. 0.] 3251
0
[0. 0. 0. 0.] 3250
0
[0. 0. 0. 0.] 3179
0
[0. 0. 0. 0.] 3178
0
[0. 0. 0. 0.] 3107
0
[0. 0. 0. 0.] 3106
0
[0. 0. 0. 0.] 3035
0
[0. 0. 0. 0.] 3034
0
[0. 0. 0. 0.] 2963
0
[0. 0. 0. 0.] 2962
0
[0. 0. 0. 0.] 2891
0
[0. 0. 0. 0.] 2890
0
[0. 0. 0. 0.] 2819
0
[0. 0. 0. 0.] 2818
0
[0. 0. 0. 0.] 2747
0
[0. 0. 0. 0.] 2746
0
[0. 0. 0. 0.] 2675
0
[0. 0. 0. 0.] 2674
0
[0. 0. 0. 0.] 2603
0
[0. 0. 0. 0.] 2602
0
[0. 0. 0. 0.] 2531
0
[0. 0. 0. 0.] 2530
0
[0. 0. 0. 0.] 2459
3
[0. 0. 0. 0.] 2460
3
[0. 0. 0. 0.] 2460
0
[0. 0. 0. 0.] 2459
0
[0. 0. 0. 0.] 2387
3
[0. 0. 0. 0.] 2316
3
[0. 0. 0. 0.] 2317
3
[0. 0. 0. 0.] 2316
3
[0. 0. 0. 0.] 2318
0
[0. 0. 0. 0.] 2317
0
[0. 0. 0. 0.] 2246
3
[0. 0. 0. 0.] 2247
3
[0. 0. 0. 0.] 2247
0
[0. 0. 0. 0.] 2246
0
[0. 0. 0. 0.] 2175
3
[0. 0. 0. 0.] 2176
3
[0. 0. 0. 0.] 2176
0
[0. 0. 0. 0.] 2175
0
[0. 0. 0. 0.] 2104
3
[0. 0. 0. 0.] 2105
3


SystemExit: 

In [126]:
Q1[2950]

array([0., 0., 0., 0.])

In [127]:
Q2[2950]

array([0., 0., 0., 0.])

In [128]:
Q3[2950]

array([0., 0., 0., 0.])

In [129]:
Q4[2950]

array([0.        , 0.        , 0.21857263, 0.        ])

In [115]:
Q[1124]

NameError: name 'Q' is not defined